In [10]:
import plotly 
plotly.tools.set_credentials_file(username='jbdtstocks', api_key='cCqpKMsmASdpYw5a5Hgc')
import plotly.plotly as py
from plotly import tools 
import plotly.graph_objs as go 

import pandas_datareader as pdr 
from datetime import datetime 
ticker_input = input("Type ticker symbols separated by one space:")
ticker_list = ticker_input.split()
tickers = [i for i in ticker_list]

for ticker in tickers:
    df = pdr.DataReader(ticker, "iex", start='1/1/2018')
    df['MACD'] = df['close'].ewm(span=12, adjust=False).mean() - df['close'].ewm(span=26, adjust=False).mean()
    df['SMA14'] = df['close'].rolling(window=14).mean()
    df['SMA50'] = df['close'].rolling(window=50).mean()
    df['Signal'] = df['MACD'].ewm(span=9, adjust=False).mean() 
    span = 4
    closing = df.close 
    delta = closing.diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.ewm(span).mean()
    roll_down = (down.abs()).ewm(span).mean()
    RS1 = roll_up / roll_down
    df['RSI'] = 100.0 - (100.0 / (1.0 + RS1))
    df['70%'] = 70
    df['30%'] = 30
    trace = go.Scatter(x=df.index, y=df.close, name='Market Price')
    sma14 = go.Scatter(x=df.index, y=df['close'].rolling(window=14).mean(), name='SMA 14')
    sma50 = go.Scatter(x=df.index, y=df['close'].rolling(window=50).mean(), name='SMA 50')
    macd = go.Scatter(x=df.index, y=df['close'].ewm(span=12, adjust=False).mean() - df['close'].ewm(span=26, adjust=False).mean(), name='MACD')
    signal = go.Scatter(x=df.index, y=df['MACD'].ewm(span=9, adjust=False).mean(), name='Signal')
    rsi = go.Scatter(x=df.index, y=df['RSI'], name='RSI')
    sev_pct = go.Scatter(x=df.index, y=df['70%'], mode='lines', name='70')
    thrty_pct = go.Scatter(x=df.index, y=df['30%'], mode='lines', name='30')

    call_list = []
    put_list = []

    if df['SMA14'].iloc[-1] > df['SMA50'].iloc[-1]:
        call_list.append(1)
    else:
        put_list.append(0)

    if df['MACD'].iloc[-1] >= 0:
        call_list.append(1)
    else:
        put_list.append(0)

    if df['MACD'].iloc[-1] > df['Signal'].iloc[-1]:
        call_list.append(1)
    else:
        put_list.append(0)

    if 40 < df['RSI'].iloc[-1] < 60:
        call_list.append(1)
    else:
        put_list.append(0)
    print('\n')
    print('#######################################################')
    print('\n')
    print('Number of Call tests that', ticker, 'passed:', len(call_list))
    print('Number of Put tests that', ticker, 'passed:', len(put_list))
    if len(call_list) >= 3:
        print(ticker, ' is a Call.')
    if len(put_list) >= 3:
        print(ticker, 'is a Put')
    print('\n')
    print('#######################################################')
    print('\n')
    data = [trace, sma14, sma50]
    fig = tools.make_subplots(rows=3, cols=1)
    fig.append_trace(trace, 1, 1)
    fig.append_trace(sma14, 1,1)
    fig.append_trace(sma50, 1,1)
    fig.append_trace(macd, 2,1)
    fig.append_trace(signal, 2,1)
    fig.append_trace(rsi, 3,1)
    fig.append_trace(sev_pct, 3,1)
    fig.append_trace(thrty_pct, 3,1)
    fig['layout'].update(title= ticker + '  ' + '1/1/2018 - Present')

py.iplot(fig, filename='AAPL_Candlestick')



Type ticker symbols separated by one space:PEP
1y


#######################################################


Number of Call tests that PEP passed: 2
Number of Put tests that PEP passed: 2


#######################################################


This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]
[ (3,1) x3,y3 ]

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jbdtstocks/0 or inside your plot.ly account where it is named 'AAPL_Candlestick'
